# 8章 頑健性と性能

有用な Python プログラムを書いたなら、 次のステップは、本番運用できる頑健なコードにすることである。プログラムを想定外の環境でも信頼できるようにすることは、正しい機能を備えることと同じように重要である。Python には、さまざまな状況下でもプログラムが頑健になるようにするのを手助けする組み込みの機能やモジュールがあります。

頑健さの1つの側面が、スケーラビリティと性能である。膨大なデータ量を扱う Python プログラムを実装する場合、コードのアルゴリズム計算量に由来する速度低下や他の種類の計算上のオーバーヘッドに出会うことがある。幸い、Python には最小限の努力で高性能を達成するのに必要なアルゴリズムとデータ構造が用意されている。


In [1]:
# Reproduce book environment
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

# Write all output to a temporary directory
import atexit
import gc
import io
import os
import tempfile

### （補足）logging

logging（[参考](https://docs.python.org/ja/3/howto/logging.html)）
- logging は、あるソフトウェアが実行されているときに起こったイベントを追跡するための手段
- 特定のイベントが発生したことを示す logging の呼び出しをコードに加えることで使用できる
- イベントは、メッセージで記述され、これに変数データ（イベントが起こる度に異なるかもしれないデータ）を加えることもできる
- イベントには、開発者がそのイベントに定めた重要性も含まれる（重要性は、レベル (level) や 重大度 (severity) とも呼ばれる）

重要性・レベル
- DEBUG
  - おもに問題を診断するときにのみ関心があるような、詳細な情報。
- INFO
  - 想定された通りのことが起こったことの確認。
- WARNING
  - 想定外のことが起こった、または問題が近く起こりそうである (例えば、'disk space low') ことの表示。
- ERROR
  - より重大な問題により、ソフトウェアがある機能を実行できないこと。
- CRITICAL
  - プログラム自体が実行を続けられないことを表す、重大なエラー。

## 項目 65 try/except/else/finally の各ブロックを活用する

Python で例外処理をしているときには、4種類の異なる場合がある。それらは、機能的には、try, except else, finally の各ブロックで取り扱える。それぞれのブロックが、複合文の中で独特の目的を果たし、役に立つようにさまざまに組み合わせられる（他の例としては「項目87 API からの呼び出し元を分離するために、ルート例外を定義する」参照）。

### finally ブロック

例外を上に(呼び出し元に) 伝えたいときには try / finally を用いるが、例外発生時にも後始末処理を実行したいことがある。よく使う try / finally の場面の1つが、ファイルハンドルを確実に閉じることである（別の、おそらくより良いやり方としては、「項目 66 contextlib と with 文を try / finally の代わりに考える」参照）。

In [2]:
# Example 1
def try_finally_example(filename):
    print('* Opening file')
    handle = open(filename, encoding='utf-8') # May raise OSError
    try:
        print('* Reading data')
        return handle.read()  # May raise UnicodeDecodeError
    finally:
        print('* Calling close()')
        handle.close()        # Always runs after try block


read メソッドで引き起こされた例外はすべて、呼び出したコードまで常に伝わるが、handle の close メソッドも finally ブロックで最初に実行される。

In [3]:
# Example 2
try:
    filename = 'random_data.txt'
    
    with open(filename, 'wb') as f:
        f.write(b'\xf1\xf2\xf3\xf4\xf5')  # Invalid utf-8
    
    data = try_finally_example(filename)
    # This should not be reached.
    import sys
    sys.exit(1)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\451854399.py", line 8, in <cell line: 2>
    data = try_finally_example(filename)
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\1123240798.py", line 7, in try_finally_example
    return handle.read()  # May raise UnicodeDecodeError
  File "c:\Users\su10_\anaconda3\envs\develop\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 0: invalid continuation byte


* Opening file
* Reading data
* Calling close()


open の呼び出しは try ブロックの前に実行する。ファイルを開くときに発生する（ファイルが存在しないときの IOError のような）例外は、finally ブロック全体をスキップする必要があるためである


In [4]:
# Example 3
try:
    try_finally_example('does_not_exist.txt')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\2999981595.py", line 3, in <cell line: 2>
    try_finally_example('does_not_exist.txt')
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\1123240798.py", line 4, in try_finally_example
    handle = open(filename, encoding='utf-8') # May raise OSError
FileNotFoundError: [Errno 2] No such file or directory: 'does_not_exist.txt'


* Opening file


### else ブロック

try / except / else を使うと、どの例外が自分のコードで扱われ、どの例外が上に伝わるかが明らかになる。try ブロックで例外が発生しなければ、else ブロックが実行されます。else ロックによって、try ブロックでのコードが最小化できて、例外の原因が分離され、読みやすさが向上する。例えば、JSON の辞書データを文字列からロードして、含まれているキー値を返したいとする。


In [5]:
# Example 4
import json

def load_json_key(data, key):
    try:
        print('* Loading JSON data')
        result_dict = json.loads(data)  # May raise ValueError
    except ValueError as e:
        print('* Handling ValueError')
        raise KeyError(key) from e
    else:
        print('* Looking up key')
        return result_dict[key]         # May raise KeyError

うまくいけば、JSON データは try ブロックで復号され、else ブロックでキー値の比較が行われる。

In [6]:
# Example 5
assert load_json_key('{"foo": "bar"}', 'foo') == 'bar'

* Loading JSON data
* Looking up key


入力データが正しい JSON 形式でないなら、json.loads による復号は ValueError を起こす。この例外は、except ブロックで捕捉されて処理される。

In [7]:
# Example 6
try:
    load_json_key('{"foo": bad payload', 'foo')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\1582434711.py", line 7, in load_json_key
    result_dict = json.loads(data)  # May raise ValueError
  File "c:\Users\su10_\anaconda3\envs\develop\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "c:\Users\su10_\anaconda3\envs\develop\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "c:\Users\su10_\anaconda3\envs\develop\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 9 (char 8)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\3943734403.py", line 3, in <cell line: 2>
    load_json_key('{"foo": bad payload', 'foo')
  File "C:\Users\su10_\AppData\L

* Loading JSON data
* Handling ValueError


キー値比較で何らかの例外が発生すると、これは try ブロックの外なので、呼び出し元まで伝播する。else 節は、try / except の後で起こることは、見た目にも、except ブロックとは異なるということを保証する。これによって、例外伝播の振る舞いが明らかになる。


In [8]:
# Example 7
try:
    load_json_key('{"foo": "bar"}', 'does not exist')
except:
    logging.exception('Expected')
else:
    assert False


ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\4147436497.py", line 3, in <cell line: 2>
    load_json_key('{"foo": "bar"}', 'does not exist')
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\1582434711.py", line 13, in load_json_key
    return result_dict[key]         # May raise KeyError
KeyError: 'does not exist'


* Loading JSON data
* Looking up key


### すべてをあわせる

すべてを1つの複合文で行いたい場合には、try / except / else / finally を使う。例えば、ファイルから、作業の記述を読み込み、 処理して、ファイルを更新する場合を考える。try ブロックを使ってファイルを読みだして処理する。except ブロックを使って、try ブロックで予期される例外を扱う。else ブロックは、ファイルを更新し、関連する例外を上に伝える。finally ブロックは、ファイルハンドルを解放する。


In [9]:
# Example 8
UNDEFINED = object()
DIE_IN_ELSE_BLOCK = False

def divide_json(path):
    print('* Opening file')
    handle = open(path, 'r+')   # May raise OSError
    try:
        print('* Reading data')
        data = handle.read()    # May raise UnicodeDecodeError
        print('* Loading JSON data')
        op = json.loads(data)   # May raise ValueError
        print('* Performing calculation')
        value = (
            op['numerator'] /
            op['denominator'])  # May raise ZeroDivisionError
    except ZeroDivisionError as e:
        print('* Handling ZeroDivisionError')
        return UNDEFINED
    else:
        print('* Writing calculation')
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)          # May raise OSError
        if DIE_IN_ELSE_BLOCK:
            import errno
            import os
            raise OSError(errno.ENOSPC, os.strerror(errno.ENOSPC))
        handle.write(result)    # May raise OSError
        return value
    finally:
        print('* Calling close()')
        handle.close()          # Always runs


成功すると、try, else, finally の各ブロックが実行される。

In [10]:
# Example 9
temp_path = 'random_data.json'

with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 10}')

assert divide_json(temp_path) == 0.1

* Opening file
* Reading data
* Loading JSON data
* Performing calculation
* Writing calculation
* Calling close()


計算結果が不当なら、try, except, finally の各ブロックが実行されるが、else ブロックは実行されない。

In [11]:
# Example 10
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 0}')

assert divide_json(temp_path) is UNDEFINED

* Opening file
* Reading data
* Loading JSON data
* Performing calculation
* Handling ZeroDivisionError
* Calling close()


JSONデータが不当なら、try ブロックが実行され、例外を送出する。finally ブロックが実行され、例外が呼び出し元まで上げられる。except, else ブロックは実行されない。

In [12]:
# Example 11
try:
    with open(temp_path, 'w') as f:
        f.write('{"numerator": 1 bad data')
    
    divide_json(temp_path)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\2445449472.py", line 6, in <cell line: 2>
    divide_json(temp_path)
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\3456276059.py", line 12, in divide_json
    op = json.loads(data)   # May raise ValueError
  File "c:\Users\su10_\anaconda3\envs\develop\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "c:\Users\su10_\anaconda3\envs\develop\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "c:\Users\su10_\anaconda3\envs\develop\lib\json\decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 1 column 17 (char 16)


* Opening file
* Reading data
* Loading JSON data
* Calling close()


この配置は、すべてのブロックが直感的に一緒に働くので、非常に有用である。
例えば、これを divide_json 関数実行と同時にハードディスクが満杯になった状況をシミュレーションする。

In [13]:
# Example 12
try:
    with open(temp_path, 'w') as f:
        f.write('{"numerator": 1, "denominator": 10}')
    DIE_IN_ELSE_BLOCK = True
    
    divide_json(temp_path)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\1279431829.py", line 7, in <cell line: 2>
    divide_json(temp_path)
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\3456276059.py", line 28, in divide_json
    raise OSError(errno.ENOSPC, os.strerror(errno.ENOSPC))
OSError: [Errno 28] No space left on device


* Opening file
* Reading data
* Loading JSON data
* Performing calculation
* Writing calculation
* Calling close()


結果データを書き換えているときに、else ブロックで例外が発生したとしても、finally ブロックは、ちゃんと実行されてファイルハンドルを閉じる。

### 覚えておくこと

- try / finally 複合文では、try ブロックで例外が発生しようとしまいと、後始末処理を実行できる。
- elseブロックは、try ブロックでのコードを最少にして、成功した場合を見た目にも try / except ブロックから区別できるようにする。
- else ブロックは、成功したtryブロックの後で、finally ブロックによる共通後始末処理の前に、追加作業を行うのに便利だ。


## 項目 66 contextlib と with文をtry/finally の代わりに考える

Python の with 文は、コードが特別なコンテキスト下で実行されていることを示すのに使われる。例えば、排他制御ロック(「項目54 スレッドにおけるデータ競合を防ぐためにLockを使う」参照)を with 文で使い、そのコードブロックがロック状態でのみ実行されることを示す。

In [14]:
# Example 1
from threading import Lock

lock = Lock()
with lock:
    # Do something while maintaining an invariant
    pass

この例は、Lock クラスが with 文で正しく動作するため、次の try / finally 構成と等価となる (try / finally については「項目 65 try/except/else/finally の各ブロックを活用する」参照)。

In [15]:
# Example 2
lock.acquire()
try:
    # Do something while maintaining an invariant
    pass
finally:
    lock.release()

この場合、with 文のほうが、try / finally 構成で繰り返し現れるコードを書く必要がなく、すべての acquire 呼び出しで対応する release を必ず実行するので優れている。
組み込みモジュール contextlib を使うとオブジェクトや関数を with 文で簡単に使えるようになる。
このモジュールは、単純な関数を with 文で使えるようにする contextmanager デコレータ を含む（「項目 26 functools.wraps を使って関数デコレータを定義する」 参照）。
これは、（標準的な）特殊メソッド `__enter__` や `__exit__` を持つ新たなクラスを定義するよりも、ずっと容易である。

例えば、コードの一部の範囲で、デバッグ用のロギングをたくさんする必要があったとする。
次のように、2つの重大度レベルでロギングを行う関数を定義する。


In [16]:
# Example 3
import logging
logging.getLogger().setLevel(logging.WARNING)

def my_function():
    logging.debug('Some debug data')
    logging.error('Error log here')
    logging.debug('More debug data')

プログラムのデフォルトのロギングレベルは WARNING なので、この関数を実行したときの画面に出るのは、エラーメッセージだけである

In [17]:
# Example 4
my_function()

ERROR:root:Error log here


In [38]:
# Example 4.1: DEBUGレベルにするとデバッグメッセージまで表示される
logging.getLogger().setLevel(logging.DEBUG)
my_function()

DEBUG:root:Some debug data
ERROR:root:Error log here
DEBUG:root:More debug data


この関数のロギングレベルを、コンテキストマネジャーを定義することで、一時的に上げることができる。
このヘルパー関数は、with ブロックでコードが実行される前に重大度レベルを上げて、その後では、重大度レベルを元に戻す。


In [18]:
# Example 5
from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield
    finally:
        logger.setLevel(old_level)

yield 式のところで、 with ブロックの内容が実行される（背景については「項目30 リストを返さずにジェネレータを返すことを考える」参照）。with ブロックで起こるすべての例外は、 yield 式で再度起こされてヘルパー関数で捕捉される（どのように動作するかは、「項目35 ジェネレータで throw による状態遷移を起こすのは避ける」参照）。

先ほどと同じロギング関数を、今度は debug_logging のコンテキストで実行する。with プロックでは、すべてのデバッグメッセージが出力されるのに対し、with ブロックの外では、デバッグメッセージを出さないことが確認できる。


In [19]:
# Example 6
with debug_logging(logging.DEBUG):
    print('* Inside:')
    my_function()

print('* After:')
my_function()

DEBUG:root:Some debug data
ERROR:root:Error log here
DEBUG:root:More debug data
ERROR:root:Error log here


* Inside:
* After:


### with ターゲットを使う

with 文に渡されるコンテキストマネジャーは、オブジェクトを返すこともある。
このオブジェクトは、複合文の as 部分のローカル変数に代入される。これにより、with ブロックで実行されるコードが、そのコンテキストと直接対話できる。

例えば、ファイルに書き込んだ後で、それを確実に正しく閉じたいとする。open を with 文に渡すことで、これを実現できる。
open は、with 文の as ターゲットとしてファイルハンドルを返し、with ブロックを抜けた後で自動でハンドルを閉じる。


In [20]:
# Example 7
with open('my_output.txt', 'w') as handle:
    handle.write('This is some data!')

この方式は、毎回ファイルハンドルを自分で開いて閉じるよりも Pythonic である。with 文の実行が抜けた後で、ファイルが実際に閉じられたことを確信できる。ファイルハンドルがオープンしている間に実行するコード量も減らすことができ、一般に優れたやり方である。

自分の関数で as ターゲットとして値を与えるために必要なのは、コンテキストマネジャーからその値を yield することだけである。例えば、コンテキストマネジャーで Logger インスタンスを取り出し、そのレベルを設定して、それを as ターゲットとして yield することで次のように定義できる。


In [21]:
# Example 8
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)

※捕捉： @contextmanager デコレータを使用して定義された関数は、with 文のコンテキスト内で使用されるときに呼び出される。この関数は、yield 文を含み、イテレータを返す。yield 文の前のコードはコンテキストの開始時に実行され、yield 文の後のコードはコンテキストの終了時に実行される。Example 8 のコードでは、@contextmanager デコレータが log_level 関数に適用されている。これにより、log_level 関数はコンテキストマネージャとして使用できるようになる。コンテキスト内の処理が実行される間、ログレベルが一時的に変更され、処理が終了した後に元のログレベルに戻される。

as ターゲットで debug のようなロギングメソッドを呼び出すと、with ブロックにおけるロギングの重大度レベルが十分低いため、出力がなされます。logging モジュールを直接使うと、デフォルト のプログラムロガーのデフォルトのロギング重大度レベルが WARNING なので、何も出力されない。

In [22]:
# Example 9
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug(f'This is a message for {logger.name}!')
    logging.debug('This will not print')

DEBUG:my-log:This is a message for my-log!


with 文を抜けた後で、'my-log' という名前の Logger でデバッグロギングメソッドを呼び出しても、デフォルトのロギング重大度が元に戻されているため、何も出力されない。エラーログメッセージは、常に出力される。


In [23]:
# Example 10
logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

ERROR:my-log:Error will print


後で、with文を更新するだけで、 使いたいロガーの名前を変えることができる。これにより、with ブロックの as ターゲットの Logger が別のインスタンスを指す。しかし、他のコードは更新する必要はない。


In [24]:
# Example 11
with log_level(logging.DEBUG, 'other-log') as logger:
    logger.debug(f'This is a message for {logger.name}!')
    logging.debug('This will not print')    # logging はデフォルトの WARNING レベルのため .debug の文は表示されない

DEBUG:other-log:This is a message for other-log!


この状態分離とコンテキストの作成とコンテキスト内における動作との切り離しが、with 文の別の利点である。


### 覚えておくこと

- with文は、try/finally ブロックのロジックを再利用して、見た目をすっきりさせる。
- 組み込みモジュール contextlib は、 with文で自分の関数を使うことを容易にする contextmanager デコレータを提供する。
- コンテキストマネジャーで yield した値は、with文のas部分に渡される。これは、コードが特別なコンテキストの元に直接アクセスできるようにするので便利である。


## 項目 67 ローカルクロックにはtimeではなくdatetime を使う

協定世界時 (Coordinated Universal Time: UTC) は、タイムゾーンとは独立した標準的な時刻の表現方式です。UTCは、コンピュータの中で、 Unixエポックからの秒数で時刻を表すのに使われています。しかし、UTC は、 人間にとっては理想的ではありません。 人間が使う時刻は、今どこにいるかに 依存する相対的なものです。 「UTCの15:00 マイナス7時」とは言わず、「8am」 とか 「正午」と言います。 プログラムで時刻を扱う場合には、おそらく、 UTC と現地のクロックとの間で時刻の変換をして、人間にとってわかりやすくしています。

Python は、タイムゾーンの変換を行うのに2種類の方法を提供しています。 古い方法は、組み込み モジュール time を用いるもので、 ひどいエラーが起こる危険性があります。 新しい方法は、組み込みモジュール datetime を用いるもので、pytz というコミュニティが作ったパッケージを使って素晴らしい仕事をします。

なぜdatetime が最良でtimeを避けるべきかを理解するには、 time と datetimeの両方を完全に
知っておく必要があります。


### time モジュール

組み込みモジュールtimeのlocaltime 関数は、UNIX タイムスタンプ (UTCによるUNIXエポックからの秒数)をホストコンピュータのタイムゾーン (著者の場合なら太平洋夏時間) に合致するロー カル時間に変換します。このローカル時間は、strftime 関数を使い人間に読みやすい形式で出力できます。


In [25]:
# Example 1
import time

now = 1552774475
local_tuple = time.localtime(now)
time_format = '%Y-%m-%d %H:%M:%S'
time_str = time.strftime(time_format, local_tuple)
print(time_str)

2019-03-17 07:14:35


反対に、人間に読みやすいローカル時間のユーザ入力から始めてUTC時間に変換する必要もよくあ ります。これをするには、時間文字列をパースする strptime 関数を使い、 mktime を呼び出してロー カル時間を UNIX タイムスタンプに変換します。

In [26]:
# Example 2
time_tuple = time.strptime(time_str, time_format)
utc_now = time.mktime(time_tuple)
print(utc_now)


1552774475.0


あるタイムゾーンのローカル時間を別のタイムゾーンの時間に変換するにはどうするのでしょうか。例えば、サンフランシスコとニューヨーク間の飛行機に乗って、ニューヨークに到着したときに、サンフランシスコが何時か調べたいとします。

time, localtime, strptime 関数の戻り値を直接扱ってタイムゾーンの変換をすることをまず 考えるかもしれません。しかし、これはまずい考えです。タイムゾーンは、その地方の法律により、常に変化します。自分で処理するのは、特に、飛行機が発着する国際的な都市を扱う場合には複雑になりすぎます。

多くのオペレーティングシステムでは、タイムゾーンの変更を自動的に管理するための設定ファイルを持っています。Python は、time モジュールを使って、プラットフォームがサポートしているならそのタイムゾーン情報を利用します。 Windowsのようなサポートされていないプラットフォームでは、time モジュールはタイムゾーン情報が得られないところがあります。次の例では、 太平洋夏時間のサンフランシスコの出発時間をパースします。


In [27]:
# Example 3
import os

if os.name == 'nt':
    print("This example doesn't work on Windows")
else:
    parse_format = '%Y-%m-%d %H:%M:%S %Z'
    depart_sfo = '2019-03-16 15:45:16 PDT'
    time_tuple = time.strptime(depart_sfo, parse_format)
    time_str = time.strftime(time_format, time_tuple)
    print(time_str)

This example doesn't work on Windows


PDTがstrptime関数でうまくいくのを見たら、自分のコンピュータでわかっている他のタイムゾーンもうまくいくだろうと考えるでしょう。残念ながら、これは間違いです。strptimeは、東部夏時間 (ニューヨークのタイムゾーン) で例外が発生します。


In [28]:
# Example 4
try:
    arrival_nyc = '2019-03-16 23:33:24 EDT'
    time_tuple = time.strptime(arrival_nyc, time_format)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_20756\2746807617.py", line 4, in <cell line: 2>
    time_tuple = time.strptime(arrival_nyc, time_format)
  File "c:\Users\su10_\anaconda3\envs\develop\lib\_strptime.py", line 562, in _strptime_time
    tt = _strptime(data_string, format)[0]
  File "c:\Users\su10_\anaconda3\envs\develop\lib\_strptime.py", line 352, in _strptime
    raise ValueError("unconverted data remains: %s" %
ValueError: unconverted data remains:  EDT


原因は、time モジュールは本質的にプラットフォーム依存であるからです。 実際の振る舞いは、元 になっているC関数がホストのオペレーティングシステムとどのように働いているかによって決まり ます。 これがtimeモジュールの機能をPython で信頼できないものにしています。 time モジュール は、複数のローカルタイムに対して適切に働くことができません。 したがって、この目的にはtimeモ ジュールを使うべきではありません。 timeを使わなければならないなら、UTCとホストコンピュータ のローカル時間との間の変換にだけ使うべきです。他の変換については、すべてdatetime モジュールを使いましょう。

### datetime モジュール

time モジュールと同様に、 datetime は現在時刻をUTCからローカル時間に変換するのに使えます。Python で時刻を表す第2の選択肢は、 組み込みモジュール datetimeのdatetime クラスです。現在時刻をUTCで測り、コンピュータのローカル時間 (太平洋夏時間) に変換します。


In [29]:
# Example 5
from datetime import datetime, timezone

now = datetime(2019, 3, 16, 22, 14, 35)
now_utc = now.replace(tzinfo=timezone.utc)
now_local = now_utc.astimezone()
print(now_local)

2019-03-17 07:14:35+09:00


datetime モジュールではローカル時間をUTCによるUNIX タイムスタンプに変換するのも簡単です。

In [30]:
# Example 6
time_str = '2019-03-16 15:14:35'
now = datetime.strptime(time_str, time_format)
time_tuple = now.timetuple()
utc_now = time.mktime(time_tuple)
print(utc_now)

1552716875.0


timeモジュールと異なり、 datetime モジュールはあるローカル時間から別のローカル時間に変換する機能は信頼できます。しかし、datetime は tzinfo クラスと関連メソッドのタイムゾーン演算の仕掛けしか提供していません。Python のデフォルトインストールで欠けているのは、UTC の他のタイムゾーンの定義です。

幸いなことに、Python パッケージインデックス (PyPI) (「項目82 コミュニティのモジュールをどこで見つけられるかを知っておく」参照) からダウンロードできる pytz モジュールが、その問題を解決します。pytz は、必要になるすべてのタイムゾーンの完全なデータベースを含んでいます。

pytz を効果的に使うには、ローカル時間をまず UTC に常に変換することです。必要な datetime 演算(オフセットなど) を UTC値に対して行います。それから、ローカル時間に変換します。

例えば、次に示すようにニューヨーク便到着時刻を UTC datetime に変換します。いくつかの呼び出しは冗長に見えるかもしれませんが、 pytz を使うときにはすべて必要です。


In [31]:
# Example 7
import pytz

arrival_nyc = '2019-03-16 23:33:24'
nyc_dt_naive = datetime.strptime(arrival_nyc, time_format)
eastern = pytz.timezone('US/Eastern')
nyc_dt = eastern.localize(nyc_dt_naive)
utc_dt = pytz.utc.normalize(nyc_dt.astimezone(pytz.utc))
print(utc_dt)

2019-03-17 03:33:24+00:00


UTC datetime を取得できたら、サンフランシスコのローカル時間に変換できます。

In [32]:
# Example 8
pacific = pytz.timezone('US/Pacific')
sf_dt = pacific.normalize(utc_dt.astimezone(pacific))
print(sf_dt)

2019-03-16 20:33:24-07:00


同じく簡単に、ネパールのローカル時間に変換できます。

In [33]:
# Example 9
nepal = pytz.timezone('Asia/Katmandu')
nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))
print(nepal_dt)


2019-03-17 09:18:24+05:45


datetime と pytz とで、ホストコンピュータがどんなオペレーティングシステムかにかかわらず、すべての環境で、このような変換が一貫して行えます。


### 覚えておくこと

- 異なるタイムゾーン間の変換にはtimeモジュールを使わない。
- 組み込みモジュール datetime をコミュニティモジュール pytz と一緒に使い、異なるタイムゾーン間の変換を変換する。
- 時刻を常に UTC で表し、表示の直前の段階でローカル時間に変換する。
